<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex06notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex06notebookC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


----
## 階層型ニューラルネットで手書き数字識別
----

手書き数字データの識別をニューラルネットワークにやらせる実験をしましょう．



----
### 準備




#### GPUを利用するようにランタイムのタイプを変更する



この notebook では，通常の Colab Notebook の動かし方では時間がかかる所があります（特に最後の実験）．次のようにしてランタイムのタイプを変更し，より高速な計算ができるようにしましょう．

1. メニューの「ランタイム」 > 「ランタイムのタイプを変更」 を選択．
1. 「ノートブックの設定」というポップアップウィンドウが開くので，「ハードウェアアクセラレータ」を「None」から「GPU」に変更し，「保存」する
1. いつもどおりコードセルを実行する．すでに実行していた場合，「以前のランタイムを削除する」というポップアップウィンドウが現れるので，「OK」を押しし，一番最初のコードセルから実行し直ます．

Colab Notebook は Linux を OS とする PC （クラウド上の仮想マシン）で実行されます．
通常は，その実行は CPU 上で行われますが，上記のように設定を変更することで，一部の計算を GPU にまかせることができるようになります（注）．

<span style="font-size: 75%">
※注: 無料でできますが，計算時間等が制限されています．
</span>

GPU (Graphical Processing Unit) というのは，PCのグラフィックスボード／ビデオカードやゲーム機等に搭載される，画像処理に特化した演算装置です．CPUのような汎用性がない代わりに，特定の処理をCPUよりずっと高速に実行できます．
この notebook では PyTorch という深層学習フレームワークを使用しますが， PyTorch ではニューラルネットの出力や学習のための計算を GPU 上で行って高速化することが簡単にできるようになっています．

#### いろいろ import

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

In [ ]:
# PyTorch 関係のほげ
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
import torchsummary

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

GPU が使えるようになっていれば，↑のセルを実行すると `cuda:0` と出力されるはずです．


In [ ]:
# 手書き数字データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
minimnist = np.load('minimnist.npz')
#datL = rv['datL'].astype(float)
#labL = rv['labL']
#datT = rv['datT'].astype(float)
#labT = rv['labT']
#print(datL.shape, labL.shape, datT.shape, labT.shape)

K = 10 # クラス数
D = minimnist['datL'].shape[1] # データの次元数 28 x 28 = 784

----
### 実験1




以前にも使ったことのある手書き数字データ（MNISTデータセットの一部）の識別をニューラルネットワークにやらせてみます．学習モデルは，中間層が一つおよび二つのニューラルネットワークと，ロジスティック回帰とします．


<img width="75%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/neuralnet5.png">

入力の次元数は784で，クラスは0から9までの10通りです．ニューラルネットワークの中間層ニューロン数は全て1000（3層では二つの中間層に1000個ずつ），それらの活性化関数は ReLU としました．出力層の活性化関数は softmax として，損失関数には交差エントロピーを用いました．

このような設定ですので，ロジスティック回帰モデルよりも2層ニューラルネットワークの方が，そして，2層よりも3層の方が，パラメータ数の多い複雑な学習モデルとなっています．

#### 関数等の定義

実験で使う関数等の定義をしておきます．

In [ ]:
# データを扱うためのクラス
#
class MMDataset(Dataset):

    def __init__(self, data, LT):
        if LT == 'L':
            self.X = data['datL'].astype(float) / 255
            self.Y = data['labL']
        else:
            self.X = data['datT'].astype(float) / 255
            self.Y = data['labT']

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float32)
        y = torch.tensor(self.Y[idx], dtype=torch.int64)
        return x, y

In [ ]:
# ネットワークの構造と入出力を定義するクラス
#
class MLP(nn.Module):

    def __init__(self, numNeurons):
        super(MLP, self).__init__()
        numLayers = len(numNeurons)
        assert numLayers >= 2
        L = []
        # 中間層
        for i in range(numLayers-2):
            L.append(nn.Linear(numNeurons[i], numNeurons[i+1]))
            L.append(nn.ReLU())
        # 出力層
        L.append(nn.Linear(numNeurons[-2], numNeurons[-1]))
        self.layers = nn.ModuleList(L)
        self.numNeurons = numNeurons

    def forward(self, X):
        X = X.reshape((-1, self.numNeurons[0]))
        for layer in self.layers:
            X = layer(X)
        return X


In [ ]:
# 学習の関数
#
def train(model, lossFunc, optimizer, dl):
    loss_sum = 0.0
    ncorrect = 0
    n = 0
    for i, (X, lab) in enumerate(dl):
        X, lab = X.to(device), lab.to(device)
        Y = model(X)           # 一つのバッチ X を入力して出力 Y を計算
        loss = lossFunc(Y, lab) # 正解ラベル lab に対する loss を計算
        optimizer.zero_grad()   # 勾配をリセット
        loss.backward()         # 誤差逆伝播でパラメータ更新量を計算
        optimizer.step()         # パラメータを更新
        n += len(X)
        loss_sum += loss.item()  # 損失関数の値
        ncorrect += (Y.argmax(dim=1) == lab).sum().item()  # 正解数

    return loss_sum/n, ncorrect/n

In [ ]:
# 損失関数や識別率の値を求める関数
#
@torch.no_grad()
def evaluate(model, lossFunc, dl):
    loss_sum = 0.0
    ncorrect = 0
    n = 0
    for i, (X, lab) in enumerate(dl):
        X, lab = X.to(device), lab.to(device)
        Y = model(X)           # 一つのバッチ X を入力して出力 Y を計算
        loss = lossFunc(Y, lab)  # 正解ラベル lab に対する loss を計算
        n += len(X)
        loss_sum += loss.item() # 損失関数の値
        ncorrect += (Y.argmax(dim=1) == lab).sum().item()  # 正解数

    return loss_sum/n, ncorrect/n

#### 実験

次のセルをそのまま実行しましょう．実行が終わるとグラフが表示されますので，その下のセルの説明を読みましょう．

In [ ]:
# データ読み込みの仕組み
dsL = MMDataset(minimnist, 'L')
dsT = MMDataset(minimnist, 'T')
dlL = DataLoader(dsL, batch_size=100, shuffle=False)
dlT = DataLoader(dsT, batch_size=100, shuffle=False)

# ネットワークモデルの定義
neurons = [784, 10]             # ロジスティック回帰
#neurons = [784, 1000, 10]       # 2層（中間層1層）の階層型ニューラルネット
#neurons = [784, 1000, 1000, 10] # 3層（中間層2層）の階層型ニューラルネット
net = MLP(neurons).to(device)

# 損失関数（交差エントロピー）
loss_func = nn.CrossEntropyLoss(reduction='sum')

# パラメータ最適化器
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

# 学習の繰り返し回数
nepoch = 50

# 学習
L = []
print('########## 実験1 ##########');
print(f'学習データ数: {len(dsL)}  テストデータ数: {len(dsT)}')
print()
print('# epoch  lossL  lossT  rateL  rateT')
for t in range(1, nepoch+1):
    lossL, rateL = train(net, loss_func, optimizer, dlL)
    lossT, rateT = evaluate(net, loss_func, dlT)
    L.append([t, lossL, lossT, rateL, rateT])
    if (t < 10) or (t % 10 == 0):
        print(f'{t}   {lossL:.5f}   {lossT:.5f}   {rateL:.4f}   {rateT:.4f}')

# 学習曲線の表示
data = np.array(L)
fig, ax = plt.subplots(1, 2, facecolor='white', figsize=(12, 4))
ax[0].plot(data[:, 0], data[:, 1], '.-', label='loss for training data')
ax[0].plot(data[:, 0], data[:, 2], '.-', label='loss for test data')
ax[0].axhline(0.0, color='gray')
ax[0].set_ylim(-0.05, 1.0)
ax[0].legend()
ax[0].set_title(f'loss (network = {neurons})')
ax[1].plot(data[:, 0], data[:, 3], '.-', label='accuracy for training data')
ax[1].plot(data[:, 0], data[:, 4], '.-', label='accuracy for test data')
ax[1].axhline(1.0, color='gray')
ax[1].set_ylim(0.8, 1.025)
ax[1].legend()
ax[1].set_title(f'accuracy (network = {neurons})')
plt.show()

# 学習後の損失と識別率
loss2, rrate = evaluate(net, loss_func, dlL)
print(f'# 学習データに対する損失: {loss2:.5f}  識別率: {rrate:.4f}')
loss2, rrate = evaluate(net, loss_func, dlT)
print(f'# テストデータに対する損失: {loss2:.5f}  識別率: {rrate:.4f}')

# torchsummary
torchsummary.summary(net, (1, 784))

上のセルをそのまま実行すると，ロジスティック回帰モデルに手書き数字の識別を学習させることができます．

```
# epoch  lossL  lossT  rateL  rateT
1   0.90272   0.44098   0.7610   0.8640
2   0.35590   0.33107   0.8924   0.9040
```
のような出力の各行の数値は，左から学習回数（単位は epoch（注）），学習データに対する損失，テストデータに対する損失，学習データの識別率（正解率），テストデータの識別率です．学習の進み方や汎化の様子を理解しやすくするため，学習中にテストデータを使って損失の値や識別率を算出しています（もちろん，テストデータはパラメータ更新には使っていません）．

<span style="font-size: 75%">
※注: 一般に，「全ての学習データを一回ずつ使ってパラメータを更新する作業」相当の学習作業を 1epoch（エポック）とします．1epoch中にパラメータの更新そのものは複数回行われることになるのが普通です．
この例では，学習データ数が5000でバッチサイズ 100 の mini-batch 学習を行っているので，1epoch中にパラメータの更新が50回行われています．
</span>



2つのグラフは，左が学習回数（横軸）に対する損失の値（縦軸）の変化，右が学習回数に対する識別率の値の変化を表します．"X for training data"（青） が学習データに対する値，"X for test data"（オレンジ） がテストデータに対する値です．

グラフの下の出力のうち，
```
# 学習データに対する損失: 0.00044  識別率: 1.0000
# テストデータに対する損失: 0.25200  識別率: 0.9410
```
の部分は，学習後のネットワークモデルに学習データおよびテストデータを入力して求めた損失と識別率の値を表します．

その下の出力のうち， `Trainable params:` の数は，学習に使用したネットワークモデルに含まれるパラメータの数を表します．

セルの内容を変更せずに再度実行すると，パラメータの初期値が変わるので，異なる結果が得られます．何度か実行し直してみましょう．

次に，ニューラルネットワークを使った実験をしましょう．上のコードセルの中に
```
# ネットワークモデルの定義
neurons = [784, 10]             # ロジスティック回帰
#neurons = [784, 1000, 10]       # 2層（中間層1層）の階層型ニューラルネット
#neurons = [784, 1000, 1000, 10] # 3層（中間層2層）の階層型ニューラルネット
```
という箇所があるはずです．いまは「ロジスティック回帰」の行の先頭に `#` がなく，その下の2行の先頭に `#` がついてコメントになっています．この `#` を付け替えることで，ネットワークモデルを切り替えることができます．2層や3層の階層型ニューラルネットでも実験してみましょう．


#### 考察

実験の結果を観察し，次のことを考えてみましょう．

- ロジスティック回帰，2層ニューラルネット，3層ニューラルネットのいずれも，学習を繰り返すと，学習データに対する損失の値は（ほぼ）単調に減少しているはずです．では，テストデータに対する損失の値はどうだろう？
- 3つのモデルを比較すると，学習を終えたネットワークの学習データに対する識別率はどれがよいだろう？テストデータに対する識別率ではどうだろう？
- 汎化，過適合，モデルのパラメータ数，といった言葉を使って上記の結果を説明してみよう

### 実験2

前の実験では，MNISTという手書き数字のデータセットから一部（学習データ5000個，テストデータ1000個）だけ抜き出したもので学習・テストを行っていました．今度は，MNISTのデータを全部使ってみましょう．学習データが60000個，テストデータが10000個あります．

次のセルを実行してみましょう．実行が終わるまでに3,4分かかります（CPUのみだと十数分）．まずは3層ニューラルネットを動かして，時間があれば他のも試せばよいです．

In [ ]:
# データ読み込みの仕組み
dsL = MNIST(root='mnist', train=True, download=True, transform=ToTensor())
dsT = MNIST(root='mnist', train=False, download=True, transform=ToTensor())
dlL = DataLoader(dsL, batch_size=100, shuffle=False)
dlT = DataLoader(dsT, batch_size=100, shuffle=False)

# ネットワークモデルの定義
#neurons = [784, 10]             # ロジスティック回帰
#neurons = [784, 1000, 10]       # 2層（中間層1層）の階層型ニューラルネット
neurons = [784, 1000, 1000, 10] # 3層（中間層2層）の階層型ニューラルネット
net = MLP(neurons).to(device)

# 損失関数（交差エントロピー）
loss_func = nn.CrossEntropyLoss(reduction='sum')

# パラメータ最適化器
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

# 学習の繰り返し回数
nepoch = 20

# 学習
L = []
print('########## 実験2 ##########');
print(f'学習データ数: {len(dsL)}  テストデータ数: {len(dsT)}')
print()
print('# epoch  lossL  lossT  rateL  rateT')
for t in range(1, nepoch+1):
    lossL, rateL = train(net, loss_func, optimizer, dlL)
    lossT, rateT = evaluate(net, loss_func, dlT)
    L.append([t, lossL, lossT, rateL, rateT])
    print(f'{t}   {lossL:.5f}   {lossT:.5f}   {rateL:.4f}   {rateT:.4f}')

# 学習曲線の表示
data = np.array(L)
fig, ax = plt.subplots(1, 2, facecolor='white', figsize=(12, 4))
ax[0].plot(data[:, 0], data[:, 1], '.-', label='loss for training data')
ax[0].plot(data[:, 0], data[:, 2], '.-', label='loss for test data')
ax[0].axhline(0.0, color='gray')
ax[0].set_ylim(-0.05, 0.5)
ax[0].legend()
ax[0].set_title(f'loss (network = {neurons})')
ax[1].plot(data[:, 0], data[:, 3], '.-', label='accuracy for training data')
ax[1].plot(data[:, 0], data[:, 4], '.-', label='accuracy for test data')
ax[1].axhline(1.0, color='gray')
ax[1].set_ylim(0.9, 1.01)
ax[1].legend()
ax[1].set_title(f'accuracy (network = {neurons})')
plt.show()

# 学習後の損失と識別率
loss2, rrate = evaluate(net, loss_func, dlL)
print(f'# 学習データに対する損失: {loss2:.5f}  識別率: {rrate:.4f}')
loss2, rrate = evaluate(net, loss_func, dlT)
print(f'# テストデータに対する損失: {loss2:.5f}  識別率: {rrate:.4f}')

# torchsummary
torchsummary.summary(net, (1, 784))

実行結果の見方は実験1と同じです．ただし，グラフの縦軸の範囲が実験1とは異なっていることに注意．

実験結果を実験1と比較すると，ニューラルネットの場合，学習データに対する識別率は実験1より実験2の方が少し低くなるものの，テストデータに対する識別率は実験2の方が高くなっているはずです．実験1と2で用いているニューラルネットの構造とパラメータ数は同じですので，この結果は，実験2の方が学習データ数が多いためパラメータ数の多いモデルがあまり過適合を起こさず，良好な汎化性能を示したのだと解釈できます．

このように，ニューラルネットのような複雑な機械学習モデルに性能を発揮させるには，学習データをなるべく多く用意することが重要となります（いろいろ考慮すべきことがあるので少々荒っぽい議論ですが...）．

### よだんだよん




#### 畳み込みニューラルネット

この授業で説明した階層型ニューラルネットでは，ある中間層とその前後の層との間の全てのニューロン同士がつながっています．このような構造の層は「全結合型」と呼ばれます．ここまで全ての層が全結合型の階層型ニューラルネットしか登場していませんが，実際には，対象とするデータや解きたい問題の性質に合わせて様々な構造を持つニューラルネットが用いられます．

全結合型とは異なる構造のニューラルネットの代表例に，「畳み込み (convolution)」という演算を行う層を持つ，「畳み込みニューラルネットワーク(Convolutional Neural Network, CNN)」と呼ばれるものがあります．画像や音声のようなデータを処理するのに向いており，全結合型よりもずっと少ないパラメータ数でより高い汎化性能が得られます．

ここでは実際に動かしてみることはしませんが，PyTorch では上記の `MLP` クラスの中身をちょろっと書き換えるだけで CNN を作ることができます．興味があれば調べて試してみるとよいでしょう．



#### 大規模モデルの不思議

上の実験で用いた3層の階層型ニューラルネットは，約180万個のパラメータを持っています．2014年の画像識別コンテスト（120万枚の学習画像を用いて1000クラスを識別するというもの）でトップクラスの性能を示した VGG16 という畳み込みニューラルネットは，約1億3千4百万個のパラメータを持っています（ takataka の授業のデモでよく動かしてみてもらっているやつです）．
最近話題の ChatGPT の裏にある，自然言語のデータを大量に学習した「大規模言語モデル(Large Language Model, LLM)」である GPT-3 は，1750億個のパラメータを持つとされています（GPT-3には複数種類あり，ChatGPT の裏にいるものは実際にはもっと大きいかもしれません）．

これだけパラメータ数が多いと過適合が深刻な問題となりそうなものですが，実際には VGG16 や GPT-3 等の近年の大規模モデルは高い汎化性能を示します．過適合を抑制する様々な技術が用いられていることや，学習データが非常に多いこともその理由と考えられますが，本質的なことはまだわかっていません．謎の解明に向けて理論的な面の研究が進められているところです．